In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

dataset = pd.read_csv(r'H:\RUET\3--2\CSE 3210- Sessional on 3209\Decision_Tree\Book.csv')

In [4]:
target_count = dataset.Buys_computer.value_counts()

total_rows=len(dataset.axes[0]) 
total_cols=len(dataset.axes[1])
str1=dataset.columns[total_cols-1]
uniq=np.unique(dataset[str1])

# Class count
count_class_0, count_class_1 = dataset.Buys_computer.value_counts()

# Divide by class
if(target_count[uniq[0]]>target_count[uniq[1]]):
    dataset_class_0 = dataset[dataset['Buys_computer'] == uniq[0]]
    dataset_class_1 = dataset[dataset['Buys_computer'] == uniq[1]]
else:
    dataset_class_0 = dataset[dataset['Buys_computer'] == uniq[1]]
    dataset_class_1 = dataset[dataset['Buys_computer'] == uniq[0]]

dataset_class_0_under = dataset_class_0.sample(count_class_1)
dataset_under = pd.concat([dataset_class_0_under, dataset_class_1], axis=0)

print('Random under-sampling:')
print(dataset_under.Buys_computer.value_counts())
X_train, X_test = train_test_split(dataset_under, test_size=0.2, random_state=15)
XX = X_train.iloc[:,0:].values

total_rows_train=len(X_train.axes[0]) 
total_cols_train=len(X_train.axes[1])
str1_train=dataset.columns[total_cols_train-1]
print(str1_train)
uniq_train=np.unique(X_train[str1_train])
print(uniq_train)
print(total_rows_train,total_cols_train)

Random under-sampling:
Yes    5
No     5
Name: Buys_computer, dtype: int64
Buys_computer
['No' 'Yes']
8 5


In [7]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entr = 0
    
    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset
    
    return total_entr


1.0


In [9]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

In [10]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting


In [11]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

Student


In [12]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of feature_value = count of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #should extend the node, so the branch is marked with ?
            
    return tree, train_data

In [13]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset


In [14]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data_m, label, class_list) #start calling recursion
    return tree

In [17]:
tree = id3(X_train, 'Buys_computer')
print(tree)

{'Student': {'Yes': {'Age': {'>40': {'Income': {'Medium': 'Yes', 'Low': 'No'}}, '<=30': 'Yes'}}, 'No': {'Age': {'>40': 'No', '31-40': 'Yes', '<=30': 'No'}}}}
